In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
from ast import literal_eval

In [5]:
from nltk.stem import SnowballStemmer

In [50]:
import difflib

In [99]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [317]:
df = pd.read_csv('credits.csv')

In [318]:
df1 = pd.read_csv('keywords.csv')
df2 = pd.read_csv('movies_metadata.csv')

C:\Users\hp\AppData\Local\Temp\ipykernel_24624\4002361806.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('movies_metadata.csv')


In [322]:
df1.info() #id has object datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [319]:
df2['id'] = pd.to_numeric(df['id']).astype('int64') #converted it to int64
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  int64  
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [10]:
df2.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [12]:
df1.columns

Index(['id', 'keywords'], dtype='object')

In [14]:
df.columns

Index(['cast', 'crew', 'id'], dtype='object')

In [315]:
sorted_df = df.sort_values('id')
sorted_df

,cast,crew,id
4342,"[{'cast_id': 3, 'character': 'Taisto Olavi Kas...","[{'credit_id': '52fe420dc3a36847f800001f', 'de...",2
12947,"[{'cast_id': 5, 'character': 'Nikander', 'cred...","[{'credit_id': '52fe420dc3a36847f8000077', 'de...",3
17,"[{'cast_id': 42, 'character': 'Ted the Bellhop...","[{'credit_id': '52fe420dc3a36847f800011b', 'de...",5
474,"[{'cast_id': 7, 'character': 'Frank Wyatt', 'c...","[{'credit_id': '52fe420dc3a36847f800023d', 'de...",6
256,"[{'cast_id': 3, 'character': 'Luke Skywalker',...","[{'credit_id': '52fe420dc3a36847f8000437', 'de...",11
...,...,...,...
45088,"[{'cast_id': 0, 'character': 'Bozena', 'credit...","[{'credit_id': '595f97249251412251083338', 'de...",465044
45283,"[{'cast_id': 2, 'character': 'Alec Beggs', 'cr...","[{'credit_id': '5975903bc3a3680cb700867b', 'de...",467731
21890,[],"[{'credit_id': '597ae87a925141364e00a73e', 'de...",468343
45408,"[{'cast_id': 0, 'character': 'Satu', 'credit_i...","[{'credit_id': '597e22f69251415d7801c74a', 'de...",468707


In [320]:
merged_df = pd.merge(df1, df2, on='id',how='inner')
merged_df = pd.merge(merged_df, df, on='id',how='inner') #merged all dataframes
merged_df=merged_df[['keywords', 'adult', 'belongs_to_collection','genres','original_language', 'original_title','production_companies',
       'production_countries','cast', 'crew']]

In [12]:
merged_df.head(2)

,keywords,adult,belongs_to_collection,genres,original_language,original_title,production_companies,production_countries,cast,crew
0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",False,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,Toy Story,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",False,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,Jumanji,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [38]:
merged_df['belongs_to_collection'][0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [42]:
merged_df['production_countries']

0        [{'iso_3166_1': 'US', 'name': 'United States o...
1        [{'iso_3166_1': 'US', 'name': 'United States o...
2        [{'iso_3166_1': 'US', 'name': 'United States o...
3        [{'iso_3166_1': 'US', 'name': 'United States o...
4        [{'iso_3166_1': 'US', 'name': 'United States o...
                               ...                        
46641               [{'iso_3166_1': 'IR', 'name': 'Iran'}]
46642        [{'iso_3166_1': 'PH', 'name': 'Philippines'}]
46643    [{'iso_3166_1': 'US', 'name': 'United States o...
46644             [{'iso_3166_1': 'RU', 'name': 'Russia'}]
46645     [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]
Name: production_countries, Length: 46646, dtype: object

In [54]:
merged_df['production_companies'][69]

"[{'name': 'A Band Apart', 'id': 59}, {'name': 'Dimension Films', 'id': 7405}, {'name': 'Los Hooligans Productions', 'id': 11705}, {'name': 'Miramax', 'id': 53009}]"

In [26]:
merged_df.columns

Index(['id', 'keywords', 'adult', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [216]:
merged_df=merged_df[['keywords', 'adult', 'belongs_to_collection','genres','original_language', 'original_title','production_companies',
       'production_countries','cast', 'crew']]

In [164]:
merged_df.head(1)

,keywords,adult,belongs_to_collection,genres,original_language,original_title,production_companies,production_countries,cast,crew
0,"[jealousy, toy, boy, friendship]",False,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,Toy Story,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."


In [217]:
features = ['cast', 'crew', 'keywords', 'genres','production_companies','production_countries']
for feature in features:
    merged_df[feature] = merged_df[feature].apply(lambda x: [] if pd.isnull(x) else literal_eval(x))

In [58]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46646 entries, 0 to 46645
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   keywords               46646 non-null  object
 1   adult                  46646 non-null  object
 2   belongs_to_collection  4569 non-null   object
 3   genres                 46646 non-null  object
 4   original_language      46635 non-null  object
 5   original_title         46646 non-null  object
 6   production_companies   46642 non-null  object
 7   production_countries   46642 non-null  object
 8   cast                   46646 non-null  object
 9   crew                   46646 non-null  object
dtypes: object(10)
memory usage: 4.9+ MB


In [81]:
 merged_df['keywords'][0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [76]:
merged_df['original_title'][5]

'Heat'

In [151]:
y = merged_df['keywords']
y[0]

[{'id': 931, 'name': 'jealousy'},
 {'id': 4290, 'name': 'toy'},
 {'id': 5202, 'name': 'boy'},
 {'id': 6054, 'name': 'friendship'},
 {'id': 9713, 'name': 'friends'},
 {'id': 9823, 'name': 'rivalry'},
 {'id': 165503, 'name': 'boy next door'},
 {'id': 170722, 'name': 'new toy'},
 {'id': 187065, 'name': 'toy comes to life'}]

In [218]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first 3. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [219]:
merged_df['keywords']=merged_df['keywords'].apply(get_list)

In [17]:
merged_df['keywords']  #keywords settled

0                                     [jealousy, toy, boy]
1        [board game, disappearance, based on children'...
2             [fishing, best friend, duringcreditsstinger]
3        [based on novel, interracial relationship, sin...
4                       [baby, midlife crisis, confidence]
                               ...                        
46641                      [wrestling, sport, silent film]
46642                                                   []
46643                     [physical abuse, sexual assault]
46644                                                   []
46645                                                   []
Name: keywords, Length: 46646, dtype: object

In [176]:
merged_df['belongs_to_collection']

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
46641                                                  NaN
46642                                                  NaN
46643                                                  NaN
46644                                                  NaN
46645                                                  NaN
Name: belongs_to_collection, Length: 46646, dtype: object

In [175]:
type(merged_df['belongs_to_collection'][0]) #problem

str

In [220]:
merged_df['belongs_to_collection'] = merged_df['belongs_to_collection'].apply(lambda x: '' if pd.isnull(x) else literal_eval(x))

In [221]:
merged_df['belongs_to_collection'] = merged_df['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) else '')
merged_df['belongs_to_collection'] #good 'belongs_to_collection'settled

0                  Toy Story Collection
1                                      
2             Grumpy Old Men Collection
3                                      
4        Father of the Bride Collection
                      ...              
46641                                  
46642                                  
46643                                  
46644                                  
46645                                  
Name: belongs_to_collection, Length: 46646, dtype: object

In [221]:
merged_df['genres'][101]

[{'id': 18, 'name': 'Drama'},
 {'id': 9648, 'name': 'Mystery'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 53, 'name': 'Thriller'}]

In [222]:
merged_df['genres'] = merged_df['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [224]:
merged_df['genres'] #good

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
46641                 [Drama, Family]
46642                         [Drama]
46643       [Action, Drama, Thriller]
46644                              []
46645                              []
Name: genres, Length: 46646, dtype: object

In [230]:
merged_df['production_companies'][2]

[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]

In [223]:
merged_df['production_companies']=merged_df['production_companies'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [224]:
merged_df['production_companies'] = merged_df['production_companies'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [225]:
merged_df['production_countries']=merged_df['production_countries'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [226]:
merged_df['production_countries'] = merged_df['production_countries'].apply(lambda x: x[:2] if len(x) >=2 else x)

In [227]:
merged_df['cast'] = merged_df['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
merged_df['cast'] = merged_df['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [228]:
merged_df['cast']

0                      [Tom Hanks, Tim Allen, Don Rickles]
1           [Robin Williams, Jonathan Hyde, Kirsten Dunst]
2               [Walter Matthau, Jack Lemmon, Ann-Margret]
3        [Whitney Houston, Angela Bassett, Loretta Devine]
4               [Steve Martin, Diane Keaton, Martin Short]
                               ...                        
46641                      [Jeanne d'Alcy, Georges Méliès]
46642                                                   []
46643          [Sridevi Kapoor, Sajal Ali, Akshaye Khanna]
46644          [Markie Adams, Roberto Aguire, Tina Arning]
46645          [Giulio Brogi, Renato Cestiè, Vito Cipolla]
Name: cast, Length: 46646, dtype: object

In [243]:
merged_df['crew'][0]

[{'credit_id': '52fe4284c3a36847f8024f49',
  'department': 'Directing',
  'gender': 2,
  'id': 7879,
  'job': 'Director',
  'name': 'John Lasseter',
  'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f4f',
  'department': 'Writing',
  'gender': 2,
  'id': 12891,
  'job': 'Screenplay',
  'name': 'Joss Whedon',
  'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f55',
  'department': 'Writing',
  'gender': 2,
  'id': 7,
  'job': 'Screenplay',
  'name': 'Andrew Stanton',
  'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f5b',
  'department': 'Writing',
  'gender': 2,
  'id': 12892,
  'job': 'Screenplay',
  'name': 'Joel Cohen',
  'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'},
 {'credit_id': '52fe4284c3a36847f8024f61',
  'department': 'Writing',
  'gender': 0,
  'id': 12893,
  'job': 'Screenplay',
  'name': 'Alec Sokolow',
  'profile_path': '/v79vlRYi94BZUQnkkyzn

In [229]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return ''

In [230]:
merged_df['director'] = merged_df['crew'].apply(get_director)

In [231]:
merged_df = merged_df.drop('crew', axis=1)

In [372]:
merged_df.head(2)

,keywords,adult,belongs_to_collection,genres,original_language,production_companies,production_countries,cast,director
0,jealousi toy boy,False,toystorycollection,animation comedy family,en,pixaranimationstudios,unitedstatesofamerica,tomhanks timallen donrickles,johnlasseter
1,boardgam disappear basedonchildren'sbook,False,,adventure fantasy family,en,tristarpictures teitlerfilm interscopecommunic...,unitedstatesofamerica,robinwilliams jonathanhyde kirstendunst,joejohnston


In [232]:
merged_df['cast'] = merged_df['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [233]:
merged_df['director'] = merged_df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
merged_df['director'] = merged_df['director'].apply(lambda x: [x,x, x]) #Mention Director 3 times to give it more weight relative to the entire cast.

In [234]:
merged_df['production_countries'] = merged_df['production_countries'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [235]:
merged_df['production_companies'] = merged_df['production_companies'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [236]:
merged_df['genres'] = merged_df['genres'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [268]:
#merged_df['belongs_to_collection'] = merged_df['belongs_to_collection'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [237]:
merged_df = merged_df.drop('original_title', axis=1)

In [238]:
merged_df['keywords'] = merged_df['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [239]:
merged_df['belongs_to_collection']=merged_df['belongs_to_collection'].str.lower().str.replace(" ", "")

In [240]:
stemmer = SnowballStemmer('english')

In [241]:
merged_df['keywords'] = merged_df['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [242]:
merged_df['director'] = merged_df['director'].apply(lambda x: x[:1] if len(x) >=1 else x)

In [243]:
features=['keywords','genres','production_companies','production_countries','cast','director']
for i in features:
    merged_df[i]=merged_df[i].apply(lambda x: ' '.join(x))

In [246]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46646 entries, 0 to 46645
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   keywords               46646 non-null  object
 1   adult                  46646 non-null  object
 2   belongs_to_collection  46646 non-null  object
 3   genres                 46646 non-null  object
 4   original_language      46646 non-null  object
 5   production_companies   46646 non-null  object
 6   production_countries   46646 non-null  object
 7   cast                   46646 non-null  object
 8   director               46646 non-null  object
dtypes: object(9)
memory usage: 3.6+ MB


In [245]:
merged_df['original_language']=merged_df['original_language'].apply(lambda x: '' if pd.isnull(x) else x)

In [247]:
merged_df['TfidfTXT']=merged_df['director']+' '+merged_df['adult']+' '+merged_df['keywords']+' '+3*merged_df['belongs_to_collection']+' '+merged_df['genres']#+' '+merged_df['production_companies']+' '+merged_df['cast']

In [248]:
merged_df['TfidfTXT']

0        johnlasseter False jealousi toy boy toystoryco...
1        joejohnston False boardgam disappear basedonch...
2        howarddeutch False fish bestfriend duringcredi...
3        forestwhitaker False basedonnovel interracialr...
4        charlesshyer False babi midlifecrisi confid fa...
                               ...                        
46641    georgesméliès False wrestl sport silentfilm  d...
46642                          robertgardner False   drama
46643    raviudyawar False physicalabus sexualassault  ...
46644                                shanraj.kehl False   
46645                                paolotaviani False   
Name: TfidfTXT, Length: 46646, dtype: object

In [249]:
vectorizer = TfidfVectorizer()

In [250]:
feature_vectors = vectorizer.fit_transform(merged_df['TfidfTXT'][:3300])

In [251]:
feature_vectors

<3300x4893 sparse matrix of type '<class 'numpy.float64'>'
	with 23371 stored elements in Compressed Sparse Row format>

In [378]:
#feature_vectors_float16 = feature_vectors.astype(np.float16)

In [252]:
similarity = cosine_similarity(feature_vectors)

In [253]:
similarity

array([[1.        , 0.03591903, 0.01676607, ..., 0.00478857, 0.00634119,
        0.02137111],
       [0.03591903, 1.        , 0.00300381, ..., 0.04766301, 0.06311695,
        0.00382885],
       [0.01676607, 0.00300381, 1.        , ..., 0.00479324, 0.00634737,
        0.02139196],
       ...,
       [0.00478857, 0.04766301, 0.00479324, ..., 1.        , 0.10071699,
        0.00610978],
       [0.00634119, 0.06311695, 0.00634737, ..., 0.10071699, 1.        ,
        0.00809077],
       [0.02137111, 0.00382885, 0.02139196, ..., 0.00610978, 0.00809077,
        1.        ]])

In [254]:
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : Toy Story


In [255]:
list_of_all_titles = df2['original_title'][:3300].tolist()

In [256]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
find_close_match

['Toy Story', 'Toy Story 2', 'Tommy Boy']

In [257]:
close_match = find_close_match[0]

In [258]:
index_of_the_movie = df2[df2.original_title == close_match].index[0]
index_of_the_movie

0

In [259]:
similarity_score[2997] #toy story 2

(2997, 0.12524485821702988)

In [260]:
merged_df['TfidfTXT'][0]

'johnlasseter False jealousi toy boy toystorycollectiontoystorycollectiontoystorycollection animation comedy family'

In [310]:
merged_df['TfidfTXT'][677]

'margarethevontrotta False anti-commun polit womandirector  drama romance'

In [311]:
df['id'][677]

40001

In [306]:
df['crew'][677]

"[{'credit_id': '52fe457ac3a36847f80c9aff', 'department': 'Production', 'gender': 2, 'id': 2147, 'job': 'Producer', 'name': 'Oren Koules', 'profile_path': None}, {'credit_id': '5735235ec3a368025100060a', 'department': 'Production', 'gender': 0, 'id': 4952, 'job': 'Casting', 'name': 'Nancy Foy', 'profile_path': '/blCkmS4dqNsbPGuQfozHE6wgWBw.jpg'}, {'credit_id': '546378a1c3a368124d0023a0', 'department': 'Writing', 'gender': 2, 'id': 7748, 'job': 'Novel', 'name': 'Cornell Woolrich', 'profile_path': '/rsiAv7XyWhQrpQecLDzsYfB1teu.jpg'}, {'credit_id': '52fe457ac3a36847f80c9af9', 'department': 'Sound', 'gender': 2, 'id': 9152, 'job': 'Original Music Composer', 'name': 'Patrick Doyle', 'profile_path': '/mo87kZiHO5gsoax80Us0S2CPmS.jpg'}, {'credit_id': '52fe457ac3a36847f80c9ad7', 'department': 'Directing', 'gender': 2, 'id': 24318, 'job': 'Director', 'name': 'Richard Benjamin', 'profile_path': '/9BQ5QcoCA7sfQzPvBpX8U4zDXqn.jpg'}, {'credit_id': '52fe457ac3a36847f80c9b17', 'department': 'Editing',

In [271]:
df2[df2['original_title']=='Toy Story']['genres']

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
Name: genres, dtype: object

In [273]:
df2[df2['original_title']=='Toy Story 2']['genres']

2997    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
Name: genres, dtype: object

In [262]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 1.0000000000000002), (1, 0.035919026459725584), (2, 0.016766065936181976), (3, 0.018859330915349373), (4, 0.01662469018652658), (5, 0.004110156429375586), (6, 0.022215084356006533), (7, 0.06976587461175522), (8, 0.003925633709791419), (9, 0.0035258145465273303), (10, 0.020768471582051286), (11, 0.022765714072427657), (12, 0.09173332047239972), (13, 0.003657956841073183), (14, 0.0037062613860202142), (15, 0.003860742315766037), (16, 0.004101459594504675), (17, 0.0189696125790692), (18, 0.016319887994020333), (19, 0.01946572656353396), (20, 0.0183863144338955), (21, 0.0038175263283160028), (22, 0.004115574083526918), (23, 0.0032140351493088505), (24, 0.0044422664829976595), (25, 0.13609552810887898), (26, 0.05991141983392091), (27, 0.003550170334347501), (28, 0.003477535017906247), (29, 0.003605411118745815), (30, 0.003362633178546181), (31, 0.0035891817347469776), (32, 0.005447419314266058), (33, 0.04848696788047049), (34, 0.0036007929479771965), (35, 0.004218781306941847), (36, 0.

In [263]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(0, 1.0000000000000002), (3024, 0.4831890279215211), (2262, 0.34159591474942996), (2996, 0.3006712680678278), (1489, 0.2221392795504276), (1738, 0.20310755819294998), (3198, 0.19929175630688822), (2453, 0.19500717541200047), (2248, 0.18121941255691856), (2599, 0.17579111922143892), (405, 0.1692889170091611), (1792, 0.16723795281971188), (3037, 0.1671133051278026), (2712, 0.16656702403782872), (240, 0.16529710346257295), (129, 0.16312985964521892), (1491, 0.1575252279013067), (1918, 0.15731825512974346), (1813, 0.15703021006335963), (991, 0.15458292296384535), (2423, 0.15325341411374208), (3035, 0.15257940712081922), (1075, 0.15115874186758435), (3270, 0.14978679244288817), (2100, 0.14838041465366178), (1959, 0.14821032790426436), (1817, 0.1481429108343605), (691, 0.146464089917996), (2505, 0.1429693196063271), (1936, 0.14076195836593156), (1018, 0.1407116393076353), (2407, 0.14066844704665532), (608, 0.14051038064230054), (2787, 0.14005048951904528), (3167, 0.1362564722212589), (25, 0

In [264]:
print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = df2[df2.index==index]['original_title'].values[0]
    if (i<20):
        print(i, '.',title_from_index)
        i+=1

Movies suggested for you : 

1 . Toy Story
2 . The Two Jakes
3 . The Money Pit
4 . End of Days
5 . The Next Step
6 . Nil by Mouth
7 . Reindeer Games
8 . Doug's 1st Movie
9 . Pink Flamingos
10 . The Wood
11 . Addams Family Values
12 . Dear Jesse
13 . Blood on the Sun
14 . The Astronaut's Wife
15 . Gumby: The Movie
16 . Frankie Starlight
17 . Batman & Robin
18 . Jane Austen's Mafia!
19 . Whatever


In [265]:
#count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
#count_matrix = count.fit_transform(merged_df['TfidfTXT'][:3300])
#cosine_sim = cosine_similarity(count_matrix, count_matrix)
#similarity_score = list(enumerate(cosine_sim[index_of_the_movie]))
#sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
#print('Movies suggested for you : \n')

#i = 1

#for movie in sorted_similar_movies:
#    index = movie[0]
#    title_from_index = df2[df2.index==index]['original_title'].values[0]
#    if (i<30):
#        print(i, '.',title_from_index)
#        i+=1
